In [1]:
import statistics 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

In [2]:
#%%writefile Read.py
#encodes the characteristics of a singular TLD
class TLD: 
    def __init__(self, indx, nC, cf, pmt, ref): 
        self.info = {'number': indx, 'responses':[nC], 'chip_factors': [cf],
                     'PMT': [pmt], 'refrence_light': [ref], 'nC_mean' : 0, 
                    'nC_pct_SD' : 0, 'CF_mean' : 0, 'CF_pct_SD' : 0,}
    
    
    def add_read(self, response, chip_factor, PMT, refrence_light): 
        info_name = ['responses', 'chip_factors', 'PMT', 'refrence_light']
        info = [response, chip_factor, PMT, refrence_light]
        for val_name, val in zip(info_name, info): 
            self.info[val_name].append(val) 

#encodes the characteristics of a singular read via storing multiple tld objects
class Comparison: 
    
    def __init__(self, nC_paths, names, types = ['Chip']): 
        self.read_names = names
        self.num_groups = len(types) 
        self.types = types
        self.TLDs = []
        self.Reads = pd.DataFrame()
        for i,nC_path in enumerate(nC_paths): 
            self.pro_data(nC_path, names[i], i+1)
        self.comp_stats()
        self.Reads.index = range(1, self.Reads.shape[0]+ 1)
    
            
    def pro_data(self, path, name, read_num):              #processes file given path, creates/adds to tld objects
        read_nc = []
        read_cf = []
        PMT_Ref = []
        ttt = 0

        with open(path, 'r') as f: 
            readouts = f.readlines()                                   # Get entire file as list of strings 
            for readout in readouts:                                   # Iterate over each readout
                parsed_readout = readout.split(',')                    # Split into cols 
                if len(parsed_readout[2].replace(' ','')[1:-1]) <= 2:  # Check for pmt / ref entry 
                    read_nc.append(float(parsed_readout[-2].strip()))
                else: 
                    PMT_Ref.append(float(parsed_readout[-2].strip()))
                                     
        #storage of data on the read basis(only nC for now):
        gsize = int(len(read_nc) / self.num_groups)                   #if have mult TLD types, calcs CF for each 
        types = [item for item in self.types for _ in range(gsize)]   #labeling what type is in dataframe
        indx = 0                                                      #type individually 
        
        for i in range(self.num_groups): 
            read_cf += self.chip_factors(read_nc[indx:indx+gsize])
            indx+=gsize
        self.Reads['TLD Type'] = types
        self.Reads[f'{name} nC'] = read_nc
        self.Reads[f'{name} CF'] = read_cf
        
        for i, nc_cf in enumerate(zip(read_nc, read_cf)):              #create list of TLD objects,data on TLD basis
            nc = nc_cf[0]
            cf = nc_cf[1]
            if (i % 50 == 0) and (i != 0):                             #indexed st correct PMT, ref assigned
                ttt += 2 
            if read_num == 1:                                          
                tld = TLD(i+1, nc, cf, PMT_Ref[ttt], PMT_Ref[ttt+1])   
                self.TLDs.append(tld)
            else: 
                self.TLDs[i].add_read(nc, cf, PMT_Ref[ttt], PMT_Ref[ttt+1])
    
    
    def chip_factors(self, nc):                                        #returns list of chip factors
        cf = [] 
        median = statistics.median(nc) 
        for tld in nc: 
            cf.append(tld/median)
        return cf
    
    
    def highlight_row(self, row):
        if row['%SD CF'] > 3:
            return ['background-color: red']*len(row)
        else:
            return ['']*len(row)

    
    def cf(self):
        bool_filter = [col for col in self.Reads.columns if 'CF' in col or col == 'TLD Type']
        cf_df = self.Reads[bool_filter]
        styled_df = cf_df.style.apply(self.highlight_row, axis = 1)
        return styled_df, cf_df[cf_df['%SD CF'] > 3], cf_df
    
        
    def comp_stats(self):
        dtypes = ['nC', 'CF']
        for dtype in dtypes:  
            bool_filter = [col for col in self.Reads.columns if dtype in col]
            filtered_df = self.Reads[bool_filter]
            means = filtered_df.mean(axis=1)
            std_devs = filtered_df.std(axis=1)
            pct_SDs = (std_devs / means) * 100
            
            self.Reads[f'Mean {dtype}'] = means
            self.Reads[f'%SD {dtype}'] = pct_SDs
            
            for i, row in self.Reads.iterrows():
                for key in self.TLDs[i].info.keys():
                    if dtype in key:
                        if 'mean' in key:
                            self.TLDs[i].info[key] = means.iloc[i]
                        if 'SD' in key:
                            self.TLDs[i].info[key] = pct_SDs.iloc[i]

In [3]:
def read_graphs(df):                              #assuming only CF portion of dataframe is passed 
    num_reads = (df.shape[1] - 3)                           #taking off stat cols, used for indx of 
    tld_nums = list(range(1,df.shape[0]+1))
    
    for indx in range(1,num_reads+1): 
        col_name = df.columns[indx]
        col = list(df.iloc[:,indx])
        read_mean = np.mean(col)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        fig.suptitle(f'Chip Factors - {col_name}', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        ax1.plot(tld_nums, col)
        ax1.set_xlabel('TLD number')
        ax1.set_ylabel('Chip Factor(raw nC / median nC)')
        ax1.axhline(y=read_mean, color = 'r', label = f'Read Mean: {read_mean}')
        ax1.legend()
        
        ax2.hist(col)
        ax2.set_xlabel('Chip Factor(raw nC / median nC)')
        ax2.set_ylabel('Frequency')
        ax2.axvline(x=read_mean, color = 'r', label = f'Read Mean: {read_mean}')
        ax2.legend()

In [7]:
############## Sensitivity Info - 0:low, 1:med, 2:high ##
#as seen from the 5500-2 / sorts (5500-3)
rod_sens_og = [1,0,2,1,2,0,1,2,1,0] #low: 35-40nC, med: 45nC, high: 53-56nC
chip_sens_og = [0,2,1,2,1,0,1,2,0,2]  #low/high from respective bin, med: 99cGy (middle of sort)


##################10 C per second reads, center pos/perp for rods
nC_paths = ['T3500_24Jun11.asc', 'T3500_24Jun13.asc', 'T3500_24Jun18.asc', 'T3500_24Jun17.asc', 'T3500_24JUN19.asc', 
            'T3500_24Jun27.asc', 'T3500_24Jul02_re_ord.asc']
names = ['3500 Jun11(R1)', '3500 Jun13(R2)', 'Jun17 Test(R3)', '3500 Jun18(R4)', '3500 Jun19(R5)',
         '3500 Jun27(R6)(Warmed Up)', '3500 Jul02(R7)(Backwards)']


gpath = ['T3500_24Jun11.asc', 'T3500_24Jun13.asc', 'T3500_24Jun18.asc', 'T3500_24JUN19.asc', 'T3500_24Jun27.asc', 
        'T3500_24Jul02_re_ord.asc']  
gname = ['3500 Jun11 (R1)', '3500 Jun13 (R2)', '3500 Jun18 (R4)', '3500 Jun19 (R5)','3500 Jun27 (R6) (Warmed Up)',
        '3500 Jul02 (R7) (Backwards)']


t3500 = Comparison(gpath, gname, ['Cube', 'Chip', 'Rod'])


##################df formatting
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)

################# create dfs 
cf_red, cf_bad, cf_all = t3500.cf()

In [8]:
################### nC info
# bf = [col for col in t3500_10C.Reads.columns if 'nC' in col or col == 'TLD Type']
# nC_df = t3500_10C.Reads[bf]
# nC_df

In [9]:
cf_red

,TLD Type,3500 Jun11 (R1) CF,3500 Jun13 (R2) CF,3500 Jun18 (R4) CF,3500 Jun19 (R5) CF,3500 Jun27 (R6) (Warmed Up) CF,3500 Jul02 (R7) (Backwards) CF,Mean CF,%SD CF
1,Cube,1.044830,1.055100,1.086697,1.054289,1.064596,1.076143,1.063609,1.458387
2,Cube,0.941416,0.958307,0.955269,1.025364,0.986998,0.999101,0.977743,3.238177
3,Cube,0.911283,0.918989,0.920470,0.915589,0.876720,0.941346,0.914066,2.302214
4,Cube,1.007907,1.019712,0.961994,0.995196,0.976882,0.994850,0.992757,2.095940
5,Cube,1.082279,1.085597,1.131142,1.197112,1.076837,1.114108,1.114512,4.090911
6,Cube,0.974163,0.977428,1.010581,1.000055,0.982252,0.989379,0.988976,1.421125
7,Cube,1.008574,0.992793,0.989788,0.990384,0.927847,1.000899,0.985047,2.937908
8,Cube,0.992093,0.937862,0.955092,0.974838,1.013002,0.960554,0.972240,2.788437
9,Cube,0.984261,1.007207,1.010212,0.999945,1.061712,1.028054,1.015232,2.645688
10,Cube,1.097212,1.132371,1.130374,1.163066,1.126073,1.108493,1.126265,2.012373
